In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import matplotlib as mpl

In [8]:
file_id = '1ATASmLwbd-sVPOJ7kChU9fumc15lRvwK'
download_url = f'https://drive.google.com/uc?id={file_id}&export=download'
df = pd.read_csv(download_url)

df.head()

,Id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain
0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/2014,0,189,...,111,189,218,1,4,4,6,1,1,0
1,1,1961,Graduation,Single,57091.0,0,0,6/15/2014,0,464,...,7,0,37,1,7,3,7,5,1,0
2,10476,1958,Graduation,Married,67267.0,0,1,5/13/2014,0,134,...,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474.0,1,1,11/5/2014,0,10,...,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,8/4/2014,0,6,...,11,0,34,2,3,1,2,7,1,0


In [20]:
data_types = df.dtypes

numeric_types = [np.int64, np.float64]
numeric_vars = data_types[data_types.isin(numeric_types)].index.tolist()
exclude_vars = ['Response', 'Id', 'Teenhome', 'Kidhome', 'Complain']
numeric_vars = list(filter(lambda x: x not in exclude_vars, numeric_vars))


# **Проверка гипотез**

В рамках блока "Проверка гипотез" первично мы хотим проверить гипотезы, что **средние значения наших переменных различаются по группам целевой переменной**. </br>

Напомним, что наша целевая переменная - **Response (отклик на маркетинговую кампанию по продаже золотого членства, которое дает 20% скидку)**.  </br>

Проверять статистические различие средних между группой тех, кто купил товар, и тех, кто не купил, - мы будет через **двухвыборочный t-test для двух независимых выборок.** </br>

Постановка гипотез выглядит следующим образом (на примере - переменной Income (доход)): </br>

**H<sub>0</sub>:** Средние доходы одинаковы у тех, кто откликнулся на кампанию, и у тех, не откликнулся.</br>

**H<sub>1</sub>:** Средние доходы по группам различаются.

Уровень статистической значимости - 95%.


Гипотезы, которые мы будет проверять с помощью t-test, относятся к количественным переменным из нашего датасета:

**H<sub>1</sub>:** **Средние общие расходы (Total_Spending)** различаются у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
**Клиенты с высокими общими расходами** могут быть **более заинтересованными** принять маркетинговое предложение **с целью экономии** в конкретной сети. [Покупка программы лояльности также может увеличить траты](https://www.mckinsey.com/capabilities/growth-marketing-and-sales/our-insights/coping-with-the-big-switch-how-paid-loyalty-programs-can-help-bring-consumers-back-to-your-brand) данного клиента впоследствии на этот бренд, что применимо и к сети.
</br>
</br>

**H<sub>2</sub>:** **Средние доходы (Income)** различаются у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
**Клиенты с более высоким доходом реже принимают золотое членство**, потому что у них **нет прямой цели экономить**, а основной оффер золотого членства - 20% cкидка, но без привилегий и VIP-подхода - важных качеств для когорт с более высоким доходом.
</br>
</br>

**H<sub>3</sub>:** **Средний год рождения(Year_Birth)** различается у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
[По опросным данным](https://www.inc.com/peter-roesler/new-study-shows-deals-promotions-affect-every-part-of-shopping-experience.html) - **миллинеалы (18-34 лет) чаще ищут промо-предложение с скидками** в Интернете перед покупкой, чем представители старшего поколения (55+). Возможно, более старшие поколения их ищут в оффлайне, но различия возрастные могут быть, поэтому проверим данную гипотезу. Также метод проведения кампании может повлиять на средний возраст принятия оффера - [с возрастом люди чаще предпочитают телефонные звонки как средство коммуникации в России](https://iom.anketolog.ru/2021/08/31/pozvoni-mne).
</br>
</br>

**H<sub>4</sub>:** **Средний количество покупок со скидками (NumDealsPurchases)** различается у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
**Заинтересованные экономить на обычных скидках**, могут быть также заинтересованы сэкономить деньги за счет **скидки клубной карты** и чаще их приобретать.
</br>
</br>

**H<sub>5</sub>:** **Среднее количество посещений сайта (NumWebVisitsMonth)**  различается у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
[Более активными интернет-пользователями являются более молодые поколения](https://www.levada.ru/2024/04/23/auditoriya-polzovatelej-interneta-sotsialnyh-setej-messendzherov-i-vpn-servisov/), а они в свою очередь [не предпочитают коммуникацию через телефонный звонок](https://iom.anketolog.ru/2021/08/31/pozvoni-mne). Это могло повлиять на предложение им клубной карты.
</br>
</br>

**H<sub>6</sub>:** **Среднее количество покупок в оффлайн-магазине (NumStorePurchases)**  не различается у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
</br>

**H<sub>7</sub>:** **Среднее количество покупок через каталог (NumCatalogPurchases)**  не различается у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
</br>

**H<sub>8</sub>:** **Среднее количество покупок в онлайн-магазине(NumWebPurchases)**  не различается у тех, кто откликнулся на кампанию, и у тех, не откликнулся.
</br>
</br>

**H<sub>9</sub>:** **Среднее количество дней с последней покупки (Recency)**  различается у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
Клиенты, совершавшие **покупки недавно, чаще соглашаются**, в то время, как д**авние клиенты могли потерять лояльность к сети** и быть незаинтересованными в предложении.
</br>
</br>

**H<sub>10,11,12,13,14,15</sub>:** **Средние расходы по различным категориям (MntWines, MntFishProducts, MntFruits, MntGoldProds, MntSweetProducts, MntMeatProducts)**  различаются у тех, кто откликнулся на кампанию, и у тех, не откликнулся. </br>
В целом **чем выше траты, чем большее желание у клиентов может быть экономить** на них. Но скорее всего **эффект этого может быть выше в категориях с высоким чеком** - Вино(MntWines) или Золотые изделия (MntGoldProds).

</br>


In [5]:
from scipy.stats import ttest_ind


In [21]:
results = []
var_list = numeric_vars


for var in var_list:
    group1 = df[df['Response'] == 1][var].dropna()
    group0 = df[df['Response'] == 0][var].dropna()

    t_stat, p_value = ttest_ind(group1, group0)

    results.append({
        'Variable':var,
        't-statistic':t_stat,
        'p-value':p_value
    })

results_df = pd.DataFrame(results)
print(results_df)

               Variable  t-statistic       p-value
0            Year_Birth     1.009072  3.130491e-01
1                Income     6.316421  3.225569e-10
2               Recency    -9.578014  2.520377e-21
3              MntWines    12.071816  1.492531e-32
4             MntFruits     5.974176  2.683116e-09
5       MntMeatProducts    11.506382  8.296291e-30
6       MntFishProducts     5.299728  1.273473e-07
7      MntSweetProducts     5.591218  2.529654e-08
8          MntGoldProds     6.681623  2.972725e-11
9     NumDealsPurchases     0.105889  9.156797e-01
10      NumWebPurchases     7.115161  1.498588e-12
11  NumCatalogPurchases    10.710356  3.866111e-26
12    NumStorePurchases     1.863631  6.250446e-02
13    NumWebVisitsMonth    -0.188598  8.504254e-01


### Выводы:
* Наша **H<sub>1</sub> подтвердилась - средние общие расходы (Total_Spending) выше у тех, кто откликнулся на кампанию, чтобы больше сэкономить.** Такие же результаты и с гипотезами по переменными расходов по категориям H<sub>10,11,12,13,14,15</sub> - средние по ним тоже статистически значимо выше у группы тех, кто оформил клубную карту.
* **Гипотеза H<sub>2</sub> подтвердилась, но в обратную сторону от наших предложений.** Средние различаются, но все же те, кто покупали клубные карты, были богаче тех, кто их себе не приобрел. Возможно, здесь есть **эффекты цены клубной карты - почти 500$**.
* **Гипотезы H<sub>7</sub> и H<sub>8</sub> не подтвердились.** В них мы не предполагали, что будет какая-то разница, так как не нашли доводов для их существования. Но все же статистическая разница в средних по переменным количества покупок в каталоге/онлайн-магазине существует
* **Гипотеза H<sub>9</sub> подтвердилась. Клиенты, совершавшие покупки недавно, чаще соглашаются.** Возможно, это связано с нашей предпоссылкой о лояльности клиента.
* **Средние статистически значимо не различаются только по четырем переменным - NumStorePurchases(H<sub>6</sub>), Year_Birth(H<sub>3</sub>), NumWebVisitsMonth (H<sub>5</sub>) и NumDealsPurchases (H<sub>4</sub>).** Это значит, что количество покупок в оффлайн-магазинах, возраст, количество посещений веб-сайта и количество покупок с скидками не различаются у тех, кто оформил клубную карту и у тех, кто ее не купил:
  - Для **NumStorePurchases(H<sub>6</sub>)** - мы не делали предпоссылок, что они будут различаться.
  - Для **Year_Birth(H<sub>3</sub>) и NumWebVisitsMonth (H<sub>5</sub>)** - основной предпоссылкой служило различие молодого и старшего поколения. При разведывательном анализе затрагиваются широкие возрастные группы, но возможно они не все репрезентативно представлены в датасете.
  - Для **NumDealsPurchases (H<sub>4</sub>)** - основной предпоссылкой являлось желание сэкономить, но возможно, данная группа пользователей и так экономит на обычных скидках, выбирая только данные товары, и они не готовы тратить 500 долларов на клубную карту для скидки на обычные товары.